In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
script_dir = os.getcwd() # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data.xlsx")
df = pd.read_excel(file_path)

In [ ]:
# Create a new column by concatenating 'context' and 'question'
df['context_question'] = df['context'] + " " + df['question']

# Display the first few rows to verify
print(df[['context', 'question', 'context_question']].head())

                                             context  \
0  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...   
1  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...   
2  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...   
3  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...   
4  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...   

                                            question  \
0  Kan de minister voor elke instelling (departem...   
1  a) Indien er geen richtlijnen bestaan, hoe wor...   
2  b) Is het een ambtenaar die in contact komt me...   
3  c) Is het een ambtenaar die niet in contact ko...   
4  d) Dringt de minister er bij zijn/haar collega...   

                                    context_question  
0  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...  
1  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...  
2  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...  
3  Ambtenaren Vlaamse overheid  -  HoofddoekWat h...  
4  Ambtenaren Vlaamse overheid  -  HoofddoekWat h..

In [7]:
# Group by 'context_question' and count unique themes
duplicates_with_diff_themes = df.groupby("context_question")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Merge back with the original dataframe to get all rows with these 'context_question'
filtered_df = df[df["context_question"].isin(duplicates_with_diff_themes["context_question"])]

# Sort the filtered DataFrame by 'context_question' for better readability
filtered_df = filtered_df.sort_values(by=["context_question", "theme"])

# Save the filtered rows to an Excel file
output_file_path = os.path.join(data_folder, "filtered_context_question_sorted.xlsx")
filtered_df.to_excel(output_file_path, index=False)

print(f"Filtered rows saved to {output_file_path}")

Filtered rows saved to c:\Users\corne\Documents\thesis-question-classification\Data\filtered_context_question_sorted.xlsx


In [5]:
"""

"""
# Groeperen op "question" en tellen hoeveel unieke thema's er zijn per vraag
duplicates_with_diff_themes = df.groupby("question")["theme"].nunique().reset_index()

# Aantal vragen die meer dan één thema hebben
num_questions_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1].shape[0]

# Print het resultaat
print(f"Aantal vragen met verschillende thema's: {num_questions_with_diff_themes}")

print("Vragen met verschillende thema's:")
print(duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1])

Aantal vragen met verschillende thema's: 320
Vragen met verschillende thema's:
                                                question  theme
301    Acht de minister het gereserveerde budget voor...      2
325    Acht de minister het vooropgestelde budget voo...      3
388    Als er grote verschillen zijn in het aantal aa...      2
418                                           Arbitrair?      2
465    Beschikt de minister over de voornaamste reden...      2
...                                                  ...    ...
19466  f) Wordt na een klantencontact standaard een b...     21
19473  g) Hoe wordt bepaald welke aspecten van de die...     21
19487      g) Wat is de stand van zaken van het project?      2
19506         h) Worden de resultaten gestandaardiseerd?     21
19590  vanaf de eerste dag van de publicatie (kenbaar...     10

[320 rows x 2 columns]


In [6]:
# Missende waarden controleren
print("\nMissende waarden per kolom:")
print(df.isnull().sum())

# Unieke waarden per kolom
print("\nUnieke waarden per kolom:")
print(df.nunique())

# 📌 3. Basis statistieken van tekst
df['text_lengte'] = df['text'].apply(lambda x: len(str(x)))  # Pas 'text' aan naar jouw kolomnaam

print("\nBeschrijving van tekstlengtes:")
print(df['text_lengte'].describe())

# Visualisatie tekstlengtes
plt.figure(figsize=(10,5))
sns.histplot(df['text_lengte'], bins=50, kde=True)
plt.title("Verdeling van Tekstlengtes")
plt.xlabel("Tekstlengte")
plt.ylabel("Frequentie")
plt.show()


Missende waarden per kolom:
context        9
question       0
statistical    0
theme          0
file_name      0
dtype: int64

Unieke waarden per kolom:
context         9967
question       19643
statistical        2
theme             37
file_name       3471
dtype: int64


KeyError: 'text'

In [ ]:
# Zet dit aan als je stopwoorden wilt gebruiken
nltk.download('stopwords')
nltk.download('punkt')

# 📌 1. Dataset inladen
file_path = "pad/naar/Grote_data.xlsx"  # Pas dit aan
df = pd.read_excel(file_path)

# 📌 2. Data verkennen
print("Dataset overzicht:")
print(df.info())
print(df.head())



# 📌 4. Woordanalyse
stopwoorden = set(stopwords.words('dutch'))  # Verander naar 'english' of andere taal indien nodig

def preprocess_text(text):
    tokens = word_tokenize(str(text).lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stopwoorden]
    return tokens

df['tokens'] = df['text'].apply(preprocess_text)

# Woordfrequenties
woorden = [woord for sublist in df['tokens'] for woord in sublist]
woord_frequenties = Counter(woorden)

print("\nTop 10 meest voorkomende woorden:")
print(woord_frequenties.most_common(10))

# Woordcloud genereren
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(woorden))
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# 📌 5. N-gram analyse (bigrammen en trigrammen)
vectorizer = CountVectorizer(ngram_range=(2,3), stop_words=list(stopwoorden))
X_ngrams = vectorizer.fit_transform(df['text'].dropna())

ngram_frequenties = Counter(dict(zip(vectorizer.get_feature_names_out(), X_ngrams.toarray().sum(axis=0))))
print("\nTop 10 meest voorkomende bigrammen/trigrammen:")
print(ngram_frequenties.most_common(10))

# 📌 6. TF-IDF Vectorisatie
tfidf = TfidfVectorizer(stop_words=list(stopwoorden))
X_tfidf = tfidf.fit_transform(df['text'].dropna())

# 📌 7. Dimensiereductie: PCA & t-SNE
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_tfidf.toarray())

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1])
plt.title("PCA Visualisatie van de Data")
plt.show()

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_tfidf.toarray())

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_tsne[:,0], y=X_tsne[:,1])
plt.title("t-SNE Visualisatie van de Data")
plt.show()
